# 🎯 Advanced Business Logic - Interview Scenarios

## **Module Overview**
**Master complex business logic implementation with advanced PySpark operations for enterprise-level data processing.**

**🎯 Difficulty**: 🔴 Advanced
**⏱️ Estimated Time**: 3-4 hours
**🎓 Learning Focus**: Complex business logic and enterprise patterns

---


## 🎯 What You Will Learn

By completing this module, you will master:
- ✅ **Data reconciliation** and comparison techniques
- ✅ **Complex hierarchical processing** (parent-child relationships)
- ✅ **Advanced filtering** and conditional logic
- ✅ **Business rule implementation** with PySpark
- ✅ **Data quality analysis** and null handling
- ✅ **Array and collection operations**

---


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Advanced Business Logic").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


# 🚀 Scenario 1: Data Reconciliation

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement data reconciliation solution in PySpark"**

### **💼 Business Context:**
Compare source and target datasets for data quality - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 1: Data Reconciliation (Source vs Target comparison)

source_data = [(1, "A"), (2, "B"), (3, "C"), (4, "D")]
target_data = [(1, "A"), (2, "B"), (4, "X"), (5, "F")]

df_source = spark.createDataFrame(source_data, ["id", "name"])
df_target = spark.createDataFrame(target_data, ["id1", "name1"])

print("Source Data:")
df_source.show()
print("Target Data:")
df_target.show()

# Method 1: SQL Approach
df_source.createOrReplaceTempView("sourcetab")
df_target.createOrReplaceTempView("targettab")

print("=== SQL Approach ===")
spark.sql("""
    SELECT COALESCE(s.id, t.id1) AS id,
           CASE
               WHEN s.name IS NULL THEN 'new in target'
               WHEN t.name1 IS NULL THEN 'new in source'
               WHEN s.name != t.name1 THEN 'mismatch'
               ELSE 'match'
           END AS status
    FROM sourcetab s
    FULL OUTER JOIN targettab t ON s.id = t.id1
    WHERE s.name != t.name1 OR s.name IS NULL OR t.name1 IS NULL
""").show()

# Method 2: DSL Approach
print("=== DSL Approach ===")
df_joined = df_source.join(df_target, df_source["id"] == df_target["id1"], "outer")

# Filter mismatched and null records
df_filtered = df_joined.filter(
    (col("name") != col("name1")) | col("name").isNull() | col("name1").isNull()
)

# Coalesce IDs and add status column
df_result = df_filtered.withColumn("id", coalesce(col("id"), col("id1")))\
    .drop("id1")\
    .withColumn("status",
        when(col("name").isNull(), "new in target")
        .when(col("name1").isNull(), "new in source")
        .when(col("name") != col("name1"), "mismatch")
        .otherwise("match")
    ).drop("name", "name1")

df_result.show()


# 🚀 Scenario 2: Salary Increment Analysis

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement salary increment analysis solution in PySpark"**

### **💼 Business Context:**
Calculate year-over-year salary changes - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 2: Calculate salary increments over time

salary_data = [
    (1, 60000, 2018), (1, 70000, 2019), (1, 80000, 2020),
    (2, 60000, 2018), (2, 65000, 2019), (2, 65000, 2020),
    (3, 60000, 2018), (3, 65000, 2019)
]

df_salaries = spark.createDataFrame(salary_data, ["empid", "salary", "year"])
df_salaries.show()

# Calculate year-over-year salary differences
window_spec = Window.partitionBy("empid").orderBy("year")

df_with_lag = df_salaries.withColumn("prev_salary", lag("salary", 1).over(window_spec))
df_with_lag.show()

df_increments = df_with_lag.withColumn(
    "salary_increment",
    when(col("prev_salary").isNull(), 0)
    .otherwise(col("salary") - col("prev_salary"))
).drop("prev_salary")

df_increments.orderBy("empid", "year").show()


# 🚀 Scenario 3: Hierarchical Data Processing

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement hierarchical data processing solution in PySpark"**

### **💼 Business Context:**
Build parent-child relationships - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 3: Build parent-child hierarchy

hierarchy_data = [
    ("A", "AA"), ("B", "BB"), ("C", "CC"),
    ("AA", "AAA"), ("BB", "BBB"), ("CC", "CCC")
]

df_hierarchy = spark.createDataFrame(hierarchy_data, ["child", "parent"])
df_hierarchy.show()

# Self-join to find child-parent-grandparent relationships
df_joined = df_hierarchy.alias("child_table")\
    .join(df_hierarchy.alias("parent_table"),
         col("child_table.child") == col("parent_table.parent"),
         "inner")\
    .select(
        col("child_table.child").alias("child"),
        col("child_table.parent").alias("parent"),
        col("parent_table.child").alias("grandparent")
    )

df_joined.show()


# 🚀 Scenario 4: Conditional Filtering

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement conditional filtering solution in PySpark"**

### **💼 Business Context:**
Remove specific values based on complex conditions - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 4: Remove maximum value from dataset

data1 = [(1,), (2,), (3,)]
data2 = [(1,), (2,), (3,), (4,), (5,)]

df1 = spark.createDataFrame(data1, ["col"])
df2 = spark.createDataFrame(data2, ["col1"])

df1.show()
df2.show()

# Find max value from df1
max_value = df1.agg(max("col")).collect()[0][0]
print(f"Maximum value to remove: {max_value}")

# Outer join and filter out the max value
df_joined = df1.join(df2, df1["col"] == df2["col1"], "outer")\
    .drop("col")

df_filtered = df_joined.filter(col("col1") != max_value).orderBy("col1")
df_filtered.show()


# 🚀 Scenario 5: Department Analytics

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement department analytics solution in PySpark"**

### **💼 Business Context:**
Find second highest values by category - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 5: Find second highest salary by department

emp_dept_data = [
    (1, "A", "A", 1000000), (2, "B", "A", 2500000),
    (3, "C", "G", 500000), (4, "D", "G", 800000),
    (5, "E", "W", 9000000), (6, "F", "W", 2000000)
]

dept_data = [("A", "AZURE"), ("G", "GCP"), ("W", "AWS")]

df_employees = spark.createDataFrame(emp_dept_data, ["emp_id", "name", "dept_id", "salary"])
df_departments = spark.createDataFrame(dept_data, ["dept_id1", "dept_name"])

df_employees.show()
df_departments.show()

# Join employees with departments
df_emp_dept = df_employees.join(df_departments,
                               df_employees["dept_id"] == df_departments["dept_id1"],
                               "inner")\
    .drop("dept_id1")

df_emp_dept.show()

# Find second highest salary per department
window_spec = Window.partitionBy("dept_id").orderBy(col("salary").desc())

df_ranked = df_emp_dept.withColumn("rank", dense_rank().over(window_spec))
df_ranked.show()

df_second_highest = df_ranked.filter(col("rank") == 2)\
    .drop("rank")\
    .select("emp_id", "name", "dept_name", "salary")

df_second_highest.show()


# 🚀 Scenario 6: Data Concatenation

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement data concatenation solution in PySpark"**

### **💼 Business Context:**
Combine multiple columns and explode results - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 6: Explode concatenated columns

multi_col_data = [("m1", "m1,m2", "m1,m2,m3", "m1,m2,m3,m4")]

df_multi = spark.createDataFrame(multi_col_data, ["col1", "col2", "col3", "col4"])
df_multi.show()

# Concatenate all columns with separator
df_concat = df_multi.withColumn(
    "concatenated",
    concat_ws("-", "col1", "col2", "col3", "col4")
).drop("col1", "col2", "col3", "col4")

df_concat.show()

# Explode the concatenated string back to individual rows
df_exploded = df_concat.selectExpr("explode(split(concatenated, '-')) as value")
df_exploded.show()


# 🚀 Scenario 7: Rating Visualization

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement rating visualization solution in PySpark"**

### **💼 Business Context:**
Create star ratings from numeric values - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 7: Create star rating visualization

# Sample food rating data (this would be defined earlier in the notebook)
food_data = [(1, "Pizza", 4), (2, "Burger", 5), (3, "Pasta", 3)]
df_food = spark.createDataFrame(food_data, ["food_id", "food_item", "rating"])

df_food.show()

# Create star rating string
df_star_rating = df_food.withColumn(
    "stars",
    expr("repeat('*', rating)")
)

df_star_rating.show()


# 🚀 Scenario 8: Family Vacation Planning

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement family vacation planning solution in PySpark"**

### **💼 Business Context:**
Complex multi-criteria matching - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 8: Find suitable countries for family vacation

# Sample family and country data
family_data = [("Smith", 5), ("Johnson", 3), ("Brown", 7)]
country_data = [("USA", 2, 6), ("Canada", 1, 4), ("Mexico", 3, 8), ("UK", 1, 3)]

df_families = spark.createDataFrame(family_data, ["name", "family_size"])
df_countries = spark.createDataFrame(country_data, ["country", "min_size", "max_size"])

df_families.show()
df_countries.show()

# Join families with suitable countries
df_family_countries = df_families.join(
    df_countries,
    (df_families["family_size"] >= df_countries["min_size"]) &
    (df_families["family_size"] <= df_countries["max_size"]),
    "inner"
).select(df_families["name"], df_families["family_size"],
         df_countries["country"], "min_size", "max_size")

df_family_countries.show()

# Count countries per family
df_country_count = df_family_countries.groupBy(df_families["name"])\
    .agg(count("*").alias("num_countries"))

df_country_count.show()

# Find family with maximum country options
window_spec = Window.orderBy(col("num_countries").desc())
df_ranked = df_country_count.withColumn("rank", row_number().over(window_spec))

df_top_family = df_ranked.filter(col("rank") == 1).drop("rank")
df_top_family.show()


# 🚀 Scenario 9: Data Quality Assessment

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement data quality assessment solution in PySpark"**

### **💼 Business Context:**
Comprehensive null analysis and imputation - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 9: Data Quality Analysis

# Sample student data with nulls
student_data = [
    (1, "John", None, "NY"),
    (2, "Jane", 25, "CA"),
    (3, "Bob", 30, None),
    (4, None, 22, "TX")
]

df_students = spark.createDataFrame(student_data, ["id", "name", "age", "city"])
df_students.show()

# Count null values per column
null_counts = df_students.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df_students.columns
])

null_counts.show()

# Remove rows with null ages
df_clean = df_students.filter(col("age").isNotNull())
df_clean.show()

# Add city data and fill missing ages with mean
city_data = [(1, "seatle", 82), (2, "london", 75), (3, "banglore", 60), (4, "boston", 90)]
df_cities = spark.createDataFrame(city_data, ["id", "city", "code"])

df_enriched = df_clean.join(df_cities, "id", "inner")\
    .select("id", "name", "age", "city", "code")

# Calculate mean age
mean_age = df_enriched.select(round(mean("age"))).collect()[0][0]
print(f"Mean age for null filling: {mean_age}")

# Fill null ages with mean
df_filled = df_enriched.na.fill({"age": mean_age})
df_filled.show()

# Filter adults (18+)
df_adults = df_filled.filter(col("age") >= 18)
df_adults.show()


# 🚀 Scenario 10: Sales Data Aggregation

## 📋 Problem Statement

### **🎯 Interview Question:**
**"Implement sales data aggregation solution in PySpark"**

### **💼 Business Context:**
Group and aggregate with collections - advanced requirement for enterprise data processing.

---


In [ ]:
# Scenario 10: Product sales analysis

# Sample sales data
sales_data = [
    ("2023-01-01", "Apple"),
    ("2023-01-01", "Banana"),
    ("2023-01-02", "Apple"),
    ("2023-01-02", "Orange")
]

df_sales = spark.createDataFrame(sales_data, ["sell_date", "product"])
df_sales.show()

# Group by date and collect products
df_daily_products = df_sales.groupBy("sell_date")\
    .agg(
        collect_set("product").alias("products"),
        size(collect_set("product")).alias("unique_products")
    )

df_daily_products.show()


# 🎉 Module Complete!

## **🏆 Congratulations!**

You have successfully completed the **Advanced Business Logic** interview scenarios module!

### **📊 What You Mastered:**
- **Enterprise-level business logic** implementation
- **Complex data reconciliation** techniques
- **Advanced analytical operations**
- **Data quality and validation** strategies
- **Hierarchical and relational** data processing

### **🚀 Next Steps:**
🎯 **You are now ready for PySpark interviews!**

Consider exploring:
- **Performance Tuning** for large-scale operations
- **Streaming Analytics** with Spark Structured Streaming
- **Machine Learning** pipelines with MLlib

---

**Happy Learning & Best of Luck in Your Interviews! 🚀**
